# #1 B를 추출 (단일키워드) 
// 블탭에 단일키워드 크롤링 후 이어서 해당 블로그의 최근글까지 크롤하는 코드
- 1-1. GUI에 추출할 키워드를 입력하고 엑셀 파일 a를 추출한다. 
- (5초후 자동실행)
- 1-2. 해당 a파일을 가지고 2번을 실행하고 b를 추출한다.
- (폴더명 : '키워드_오늘날짜')

In [1]:
import PySimpleGUI as sg
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
from openpyxl import Workbook, load_workbook
from openpyxl.styles import Font, Alignment, PatternFill, Border, Side
from openpyxl.utils.dataframe import dataframe_to_rows
import os
from datetime import datetime
import threading

def get_user_input():
    while True:
        layout = [
            [sg.Text('검색어를 입력하세요:', size=(18, 1)), sg.InputText(size=(35, 1), key='SEARCH_QUERY')],
            [sg.Text('가져올 글의 수:', size=(18, 1)), sg.Radio('15개', "RADIO1", default=True, key='15', enable_events=True),
             sg.Radio('30개', "RADIO1", default=False, key='30', enable_events=True),
             sg.Radio('50개', "RADIO1", default=False, key='50', enable_events=True), 
             sg.Radio('100개', "RADIO1", default=False, key='100', enable_events=True)],
            [sg.Text(' '*36), sg.Text('또는 직접 입력:', size=(11, 1)), sg.InputText(size=(20, 1), key='CUSTOM_NUM', enable_events=True)],
            [sg.Column([
                [sg.Submit(size=(10, 1), key='SUBMIT'), sg.Cancel(size=(10, 1))]
            ], justification='right', element_justification='right')]
        ]

        window = sg.Window('네이버 블로그 검색', layout)
        errors = []

        while True:
            event, values = window.read()
            if event in (sg.WIN_CLOSED, 'Cancel'):
                window.close()
                return None, None

            if event in ['15', '30', '50', '100']:
                window['CUSTOM_NUM'].update('')

            if event == 'CUSTOM_NUM' and values['CUSTOM_NUM']:
                for key in ['15', '30', '50', '100']:
                    window[key](value=False)

            if event == 'SUBMIT':
                errors.clear()
                search_query = values['SEARCH_QUERY'].strip()
                custom_num = values['CUSTOM_NUM'].strip()
                selected_num = [key for key in ['15', '30', '50', '100'] if values[key]]

                if not search_query:
                    errors.append('검색어를 입력해야 합니다.')

                if custom_num and selected_num:
                    errors.append('직접 입력과 선택 옵션 중 하나만 사용해주세요.')

                if custom_num:
                    try:
                        num_of_posts = int(custom_num)
                    except ValueError:
                        errors.append('직접 입력한 수는 유효한 숫자여야 합니다.')
                elif selected_num:
                    num_of_posts = int(selected_num[0])
                else:
                    errors.append('글 수를 입력하거나 선택해주세요.')

                if errors:
                    sg.popup_error("\n".join(errors))
                    continue
                window.close()
                return search_query, num_of_posts

def setup_webdriver():
    options = Options()
    options.headless = False
    return webdriver.Chrome(options=options)

def scrape_naver_blogs(search_query, num_of_posts):
    driver = setup_webdriver()
    driver.get("https://www.naver.com")
    time.sleep(1)

    driver.find_element(By.ID, "query").send_keys(search_query)
    driver.find_element(By.ID, "search-btn").click()
    time.sleep(1)

    driver.find_element(By.LINK_TEXT, "블로그").click()
    time.sleep(2)

    try:
        while True:
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            time.sleep(2)
            html = BeautifulSoup(driver.page_source, 'html.parser')
            posts = html.find_all('a', class_='title_link')
            if len(posts) >= num_of_posts or html.find('span', class_='bl_tit'):
                break

        name_links = html.find_all('a', class_='name')[:num_of_posts]
        title_links = posts[:num_of_posts]
        date_links = html.find_all('span', class_='sub')[:num_of_posts]

        data = []
        for name_link, title_link, date_link in zip(name_links, title_links, date_links):
            url = title_link['href']
            email = ''
            if "https://blog.naver.com/" in url:
                username = url.split('/')[3]
                email = f"{username}@naver.com"
            elif "https://adcr.naver.com/" in url:
                email = "파워콘텐츠"
            elif "https://post.naver.com/" in url:
                email = "포스트"

            data.append((name_link.text.strip(), "", title_link.text.strip(), date_link.text.strip(), url, email))

        df = pd.DataFrame(data, columns=['NAME', '2차키워드', '제목', '게시일', 'URL', '이메일'])
        return df
    except Exception as e:
        sg.popup_error('오류 발생:', str(e))
        return None
    finally:
        driver.quit()

def create_folder(search_query):
    current_date = datetime.now().strftime("%y%m%d")
    folder_name = f"{search_query}_{current_date}"
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
    return folder_name

def save_to_excel(df, folder_name, filename):
    file_path = os.path.join(folder_name, filename)
    wb = Workbook()
    ws = wb.active

    for r in dataframe_to_rows(df, index=False, header=True):
        ws.append(r)

    ws.column_dimensions['A'].width = 16
    ws.column_dimensions['B'].width = 16
    ws.column_dimensions['C'].width = 70
    ws.column_dimensions['D'].width = 16
    ws.column_dimensions['E'].width = 8
    ws.column_dimensions['F'].width = 24

    center_alignment = Alignment(horizontal='center')
    for cell in ws["D"] + ws["F"]:
        cell.alignment = center_alignment

    header_font = Font(bold=True)
    header_fill = PatternFill(start_color="B7DEE8", end_color="B7DEE8", fill_type="solid")
    thin_border = Border(left=Side(style='thin'), right=Side(style='thin'), top=Side(style='thin'), bottom=Side(style='thin'))

    for cell in ws[1]:
        cell.font = header_font
        cell.alignment = center_alignment
        cell.fill = header_fill
        cell.border = thin_border

    gray_fill = PatternFill(start_color="ECECEC", end_color="ECECEC", fill_type="solid")
    for row in ws.iter_rows(min_row=2, max_row=ws.max_row):
        if row[5].value in ['파워콘텐츠', '포스트']:
            for cell in row:
                cell.fill = gray_fill

    ws.auto_filter.ref = ws.dimensions
    wb.save(file_path)
    return file_path

def extract_blog_data(file_path):
    wb = load_workbook(file_path)
    ws = wb.active
    urls, names = [], []
    for row in range(2, ws.max_row + 1):  # Start from 2 to skip header
        url = ws['E' + str(row)].value
        name = ws['A' + str(row)].value
        if url and url.startswith("https://blog.naver.com/"):
            blog_id = url.split('/')[3]
            urls.append(f"https://blog.naver.com/PostList.naver?blogId={blog_id}&skinType=&skinId=&from=menu")
            names.append(name)
        time.sleep(1)
    return urls, names

def set_15_line_view(driver):
    try:
        select_box = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'a.btn_select.pcol2._ListCountToggle._returnFalse'))
        )
        select_box.click()
        time.sleep(1)
        thirty_lines_option = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[@data-value='30']"))
        )
        thirty_lines_option.click()
        time.sleep(2)
    except:
        try:
            open_list_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, 'btn_openlist.pcol2._toggleTopList._returnFalse'))
            )
            open_list_button.click()
            time.sleep(1)
            
            select_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, 'a.btn_select.pcol2._ListCountToggle._returnFalse'))
            )
            select_button.click()
            time.sleep(1)
            thirty_lines_option = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//a[@data-value='30']"))
            )
            thirty_lines_option.click()
            time.sleep(2)
        except:
            print("Failed to set 30-line view")

def scrape_blog_data(driver, url):
    driver.get(url)
    time.sleep(5)
    set_15_line_view(driver)
    try:
        WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'table.blog2_list'))
        )
        posts = driver.find_elements(By.CSS_SELECTOR, 'table.blog2_list tbody tr')[:30]
        data = []
        for post in posts:
            title_elements = post.find_elements(By.CSS_SELECTOR, 'span.ell2.pcol2')
            date_elements = post.find_elements(By.CSS_SELECTOR, 'div.wrap_td span.date.pcol2')
            if title_elements and date_elements:
                title = title_elements[0].text
                date = date_elements[0].text
                data.append((title, date))
    except Exception as e:
        print(f"Error processing {url}: {str(e)}")
        data = []
    return data

def save_detailed_data_to_excel(urls, names, data_list, folder_name, filename):
    file_path = os.path.join(folder_name, filename)
    wb = Workbook()
    ws = wb.active
    ws.append(["NAME", "2차 키워드", "글 제목", "작성일"])

    header_font = Font(bold=True)
    header_fill = PatternFill(start_color="FCD5B4", end_color="FCD5B4", fill_type="solid")
    center_alignment = Alignment(horizontal='center', vertical='center')
    thin_border = Border(left=Side(style='thin'), right=Side(style='thin'), top=Side(style='thin'), bottom=Side(style='thin'))
    grey_fill = PatternFill(start_color="F0F0F0", end_color="F0F0F0", fill_type="solid")
    
    ws.column_dimensions['A'].width = 16
    ws.column_dimensions['B'].width = 16
    ws.column_dimensions['C'].width = 70
    ws.column_dimensions['D'].width = 16

    for cell in ws[1]:
        cell.font = header_font
        cell.alignment = center_alignment
        cell.fill = header_fill
        cell.border = thin_border

    for index, (url, name, data) in enumerate(zip(urls, names, data_list)):
        for title, date in data:
            row = [name, "", title, date]
            ws.append(row)
            if index % 2 == 1:
                for cell in ws[ws.max_row]:
                    cell.fill = grey_fill

    wb.save(file_path)
    return file_path

    wb.save(filename)

def auto_close_popup(window):
    time.sleep(5)
    window.write_event_value('-TIMEOUT-', None)

def main():
    search_query, num_of_posts = get_user_input()
    if search_query is None or num_of_posts is None:
        return

    folder_name = create_folder(search_query)

    df = scrape_naver_blogs(search_query, num_of_posts)
    if df is None:
        return

    initial_filename = f"{search_query}_naver_blogs.xlsx"
    initial_file_path = save_to_excel(df, folder_name, initial_filename)
    
    # 수정된 부분: 5초 후 자동으로 닫히는 팝업 창
    popup_layout = [[sg.Text(f'파일이 저장되었습니다: {initial_file_path}')],
                    [sg.Button('OK')]]
    popup_window = sg.Window('알림', popup_layout, finalize=True)
    
    # 5초 후 자동으로 창을 닫는 스레드 시작
    threading.Thread(target=auto_close_popup, args=(popup_window,), daemon=True).start()
    
    while True:
        event, values = popup_window.read()
        if event in (sg.WIN_CLOSED, 'OK', '-TIMEOUT-'):
            break
    
    popup_window.close()

    # Now, use this file for the second part of the script
    urls, names = extract_blog_data(initial_file_path)
    
    driver = setup_webdriver()
    data_list = [scrape_blog_data(driver, url) for url in urls]
    driver.quit()

    detailed_filename = f"{search_query}_detailed_blog_data.xlsx"
    detailed_file_path = save_detailed_data_to_excel(urls, names, data_list, folder_name, detailed_filename)
    sg.popup(f'상세 데이터 파일이 저장되었습니다: {detailed_file_path}')

if __name__ == "__main__":
    main()

Exception in thread Thread-8 (auto_close_popup):
Traceback (most recent call last):
  File "c:\Program Files\Python310\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "C:\Users\USER\AppData\Roaming\Python\Python310\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Program Files\Python310\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\USER\AppData\Local\Temp\ipykernel_3692\3607581283.py", line 280, in auto_close_popup
  File "c:\Program Files\Python310\lib\site-packages\PySimpleGUI\PySimpleGUI.py", line 12057, in write_event_value
    self.TKroot.tk.willdispatch()  # brilliant bit of code provided by Giuliano who I owe a million thank yous!
AttributeError: 'NoneType' object has no attribute 'tk'


# #2 C를 추출 (수동) goyatools.com/tool1
- [input : 키워드_detailded_blog.xlsx]
- [output : processed_data.xlsx]

- 2-1. 고야툴즈에서 <2차 키워드를 추출>한다. (수동) C
- 2-2. 이슈1)_c파일 추출하고 난뒤. 첫행 위에 숫자 제거해야되는 이슈가 있음 
- 2-3. 이슈2)_웹 상에서 더블클릭하면 수정 바로 되도록 코드 수정


# #3 E를 추출 
- [input : processed_data.xlsx] //*인풋 엑셀은 첫 칼럼이 블로거이름이 와야된다. 아마 a열로 옮겨놔야할 것, 첫행이 헤더이고 a2부터 읽기시작함
- [output : processed_data_merged_20240904 ]
- 3-1. <c>파일 : 저장된 2차 키워드로 criteria를 추출한다. D를 추출한다.
- 3-2. D의 시트를 하나로 합쳐서 <E>를 추출한다.


In [9]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
from tkinter import Tk, filedialog
from openpyxl.styles import Font, Alignment, PatternFill, Border, Side
import openpyxl
from openpyxl.utils import get_column_letter
import os
from copy import copy
from datetime import datetime

def get_excel_file_path():
    root = Tk()
    root.withdraw()
    file_path = filedialog.askopenfilename(title='DB 엑셀 파일 선택', filetypes=[("Excel files", "*.xlsx *.xls")])
    return file_path

def load_keywords_from_excel(file_path):
    df = pd.read_excel(file_path, header=0)
    print(df.columns)
    if df.columns[0] not in df or df.columns[1] not in df:
        print("필요한 열이 엑셀 파일에 없습니다.")
        return pd.DataFrame()
    return df.dropna(subset=[df.columns[1]])

def save_results_to_excel(data_dict, keyword_index, input_file_path):
    base_name = os.path.splitext(os.path.basename(input_file_path))[0]
    date_str = datetime.now().strftime("%Y%m%d")
    output_path = os.path.join(os.path.dirname(input_file_path), f"{base_name}_criteria_{date_str}.xlsx")
    
    with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
        for sheet_name, records in data_dict.items():
            df = pd.DataFrame(records, columns=['2차키워드', '검색량', 'NAME', '제목', '게시일', 'URL', '이메일주소', '상위노출여부'])
            df.to_excel(writer, sheet_name=sheet_name, index=False)
            worksheet = writer.sheets[sheet_name]

            # 열 너비 설정
            column_widths = {'A': 16, 'B': 16, 'C': 16, 'D': 70, 'E': 16, 'F': 8, 'G': 24, 'H': 16}
            for col, width in column_widths.items():
                worksheet.column_dimensions[col].width = width

            # 스타일 설정
            red_bold_font = Font(bold=True, color="FF0000")
            grey_fill = PatternFill(start_color='EAEAEA', end_color='EAEAEA', fill_type="solid")
            standard_border = Border(left=Side(style='thin'), right=Side(style='thin'),
                                     top=Side(style='thin'), bottom=Side(style='thin'))

            for row in range(2, worksheet.max_row + 1):
                for col in range(1, 9):  # A to H
                    cell = worksheet.cell(row=row, column=col)
                    cell.border = standard_border
                    
                    # 짝수번째 키워드 색상 적용
                    if keyword_index[df.iloc[row-2, 0]] % 2 == 1:
                        cell.fill = grey_fill
                    
                    # C열에 시트 제목과 동일한 내용이 있다면
                    if worksheet.cell(row=row, column=3).value == sheet_name:
                        cell.font = red_bold_font
                        if col == 8:  # H열에 "criteria" 입력
                            worksheet.cell(row=row, column=8).value = "criteria"
                
            # 헤더 스타일
            for col in range(1, 9):
                cell = worksheet.cell(row=1, column=col)
                cell.font = Font(bold=True, color='FFFFFF')
                cell.fill = PatternFill(start_color='CCC0DA', end_color='CCC0DA', fill_type="solid")
                cell.alignment = Alignment(horizontal='center', vertical='center')
                cell.border = Border(left=Side(style='thin'), right=Side(style='thin'),
                                     top=Side(style='thin'), bottom=Side(style='thin'))

            # 필터 추가
            worksheet.auto_filter.ref = "A1:H" + str(worksheet.max_row)

    print(f"데이터가 성공적으로 저장되었습니다: {output_path}")
    return output_path

def copy_cell(source_cell, target_cell):
    target_cell.value = source_cell.value
    if source_cell.has_style:
        target_cell.font = copy(source_cell.font)
        target_cell.border = copy(source_cell.border)
        target_cell.fill = copy(source_cell.fill)
        target_cell.number_format = source_cell.number_format
        target_cell.protection = copy(source_cell.protection)
        target_cell.alignment = copy(source_cell.alignment)

def copy_sheet(source_sheet, target_sheet):
    for row in source_sheet.iter_rows():
        for cell in row:
            new_cell = target_sheet.cell(row=cell.row, column=cell.column)
            copy_cell(cell, new_cell)

    # 열 너비 복사
    for col in source_sheet.column_dimensions:
        target_sheet.column_dimensions[col].width = source_sheet.column_dimensions[col].width

    # 행 높이 복사
    for row in source_sheet.row_dimensions:
        target_sheet.row_dimensions[row].height = source_sheet.row_dimensions[row].height

def merge_excel_sheets_with_formatting(input_file, output_file):
    # 입력 워크북 열기
    source_wb = openpyxl.load_workbook(input_file)
    
    # 새 워크북 생성
    target_wb = openpyxl.Workbook()
    target_sheet = target_wb.active
    target_sheet.title = "Merged"

    # 첫 번째 시트의 헤더 복사
    first_sheet = source_wb.worksheets[0]
    copy_sheet(first_sheet, target_sheet)

    # 나머지 시트의 데이터 복사 (헤더 제외)
    row_offset = target_sheet.max_row
    for sheet in source_wb.worksheets[1:]:
        for row in sheet.iter_rows(min_row=2):  # 헤더 제외
            row_offset += 1
            for cell in row:
                new_cell = target_sheet.cell(row=row_offset, column=cell.column)
                copy_cell(cell, new_cell)

    # 결과 저장
    target_wb.save(output_file)
    print(f"모든 시트가 서식을 유지한 채 {output_file}로 합쳐졌습니다.")

# 메인 실행 부분
if __name__ == "__main__":
    file_path = get_excel_file_path()
    if not file_path:
        print("파일이 선택되지 않았습니다.")
        exit()

    df_keywords = load_keywords_from_excel(file_path)
    if df_keywords.empty:
        print("키워드를 불러오는 데 실패했습니다.")
        exit()

    options = Options()
    options.headless = False
    driver = webdriver.Chrome(options=options)

    data_dict = {}
    keyword_index = {}

    try:
        for index, row in df_keywords.iterrows():
            group_name, keyword = row[df_keywords.columns[0]], row[df_keywords.columns[1]]
            if keyword not in keyword_index:
                keyword_index[keyword] = len(keyword_index)
            driver.get('https://www.naver.com')
            time.sleep(1)

            driver.find_element(By.ID, "query").send_keys(keyword)
            driver.find_element(By.ID, "search-btn").click()
            time.sleep(1)

            driver.find_element(By.LINK_TEXT, '블로그').click()
            time.sleep(2)

            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            time.sleep(2)

            html = BeautifulSoup(driver.page_source, 'html.parser')
            name_links = html.find_all('a', class_='name')[:15]
            title_links = html.find_all('a', class_='title_link')[:15]
            date_links = html.find_all('span', class_='sub')[:15]

            data = []
            for name_link, title_link, date_link in zip(name_links, title_links, date_links):
                url = title_link['href']
                email = ""
                if "https://blog.naver.com/" in url:
                    username = url.split('/')[3]
                    email = f"{username}@naver.com"
                elif "https://adcr.naver.com/" in url:
                    email = "파워콘텐츠"
                elif "https://post.naver.com/" in url:
                    email = "포스트"

                data.append([keyword, '', name_link.text.strip(), title_link.text.strip(), date_link.text.strip(), url, email, ''])

            if group_name in data_dict:
                data_dict[group_name].extend(data)
            else:
                data_dict[group_name] = data
    finally:
        driver.quit()

    if data_dict:
        # 1차 저장: criteria 파일
        base_name = os.path.splitext(os.path.basename(file_path))[0]
        date_str = datetime.now().strftime("%Y%m%d")
        criteria_file = os.path.join(os.path.dirname(file_path), f"{base_name}_criteria_{date_str}.xlsx")
        criteria_file = save_results_to_excel(data_dict, keyword_index, criteria_file)
        
        print("5초 후 파일 병합을 시작합니다...")
        time.sleep(5)

        # 2차 저장: merged 파일
        merged_file = os.path.join(os.path.dirname(file_path), f"{base_name}_merged_{date_str}.xlsx")
        merge_excel_sheets_with_formatting(criteria_file, merged_file)

Index(['Unnamed: 0', '2차 키워드', '글 제목', '작성일', 'Unnamed: 4'], dtype='object')
데이터가 성공적으로 저장되었습니다: C:/Users/USER/Desktop/블탭검색기/보풀제거기_241128\processed_data (4)_criteria_20241128_criteria_20241128.xlsx
5초 후 파일 병합을 시작합니다...
모든 시트가 서식을 유지한 채 C:/Users/USER/Desktop/블탭검색기/보풀제거기_241128\processed_data (4)_merged_20241128.xlsx로 합쳐졌습니다.


# #4 인원선정(수동)
- [input : processed_data_merged_20240904 ]
- [output : 기본 같이 저장 ]
-
- 4-1. E파일을 바탕으로 인원선정을 시작한다.


# #5 1차 필터
- [input : processed_data_merged_20240904 ]
- [output : filter01_processed_data_merged_20240904 ]

- 5-1. 1차 필터를 한다.

In [ ]:
import pandas as pd
import tkinter as tk
from tkinter import filedialog
import os
import re

def select_file():
    root = tk.Tk()
    root.withdraw()
    file_path = filedialog.askopenfilename(filetypes=[("Excel files", "*.xlsx *.xls")])
    return file_path

def process_url(url):
    return re.sub(r'/\d+$', '', url)

# 파일 선택
input_file = select_file()
if not input_file:
    print("파일이 선택되지 않았습니다. 프로그램을 종료합니다.")
    exit()

# 엑셀 파일 읽기 (서식 유지)
with pd.ExcelFile(input_file) as xls:
    df = pd.read_excel(xls, sheet_name=0, engine='openpyxl')

# URL 형식 수정 (F열, index 5)
df.iloc[:, 5] = df.iloc[:, 5].apply(lambda x: ' '.join([process_url(url) for url in str(x).split()]))

# G열(index 6)의 중복을 확인하고 첫 번째 발견된 항목만 유지
df_no_duplicates = df.drop_duplicates(subset=df.columns[6], keep='first')

# index1(B열) 삭제
df_no_duplicates = df_no_duplicates.drop(df_no_duplicates.columns[1], axis=1)

# 새 파일명 생성
dir_path = os.path.dirname(input_file)
file_name = os.path.basename(input_file)
new_file_name = f"filter01_{file_name}"
output_file = os.path.join(dir_path, new_file_name)

# 결과를 새 엑셀 파일로 저장 (서식 유지)
with pd.ExcelWriter(output_file, engine='openpyxl', mode='w') as writer:
    df_no_duplicates.to_excel(writer, index=False, sheet_name='Sheet1')

print(f"처리 전 행 수: {len(df)}")
print(f"처리 후 행 수: {len(df_no_duplicates)}")
print(f"제거된 중복 행 수: {len(df) - len(df_no_duplicates)}")
print(f"결과가 다음 파일에 저장되었습니다: {output_file}")

#6 지수검색
- [input : filter01_processed_data_merged_20240904 ]
- [output : 지수추출_filter01_processed_data_merged_20240904 ]


In [8]:
import pandas as pd
import tkinter as tk
from tkinter import filedialog
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import time
from openpyxl import load_workbook
from openpyxl.styles import Font, PatternFill
from openpyxl.worksheet.filters import FilterColumn

# GUI 초기화
root = tk.Tk()
root.withdraw()


# 입력 엑셀 파일 선택
excel_path = filedialog.askopenfilename(title="블로그 URL이 포함된 엑셀 파일 선택", filetypes=[("Excel files", "*.xlsx *.xls")])

# 저장 경로 미리 선택
save_path = filedialog.asksaveasfilename(title="결과를 저장할 엑셀 파일 선택", defaultextension=".xlsx", filetypes=[("Excel files", "*.xlsx *.xls")])

# 엑셀 파일 불러오기
df = pd.read_excel(excel_path, usecols=[4], header=0)  # E열에서 데이터 읽기, 첫 행은 헤더

# Selenium 웹드라이버 설정
options = Options()
options.headless = False
driver = webdriver.Chrome(options=options)


# Blogdex 로그인 페이지로 이동
url = "https://blogdex.space/login?from=/blog-index"
driver.get(url)
time.sleep(3)  # 로그인 이후 타임슬립 3초

# 카카오톡 로그인 버튼 클릭
driver.find_element(By.CLASS_NAME, "border-primary").click()
time.sleep(0.3)
driver.find_element(By.XPATH, "//button[contains(., '카카오톡')]").click()
time.sleep(3)  # 로그인 이후 타임슬립 3초

# 로그인 정보 입력
driver.find_element(By.CLASS_NAME, "tf_g").send_keys("01048460380")
time.sleep(1)
password_field = driver.find_elements(By.CLASS_NAME, "tf_g")[1]
password_field.send_keys('@gusqls2')
time.sleep(0.3)
driver.find_element(By.CSS_SELECTOR, ".btn_g.highlight.submit").click()

# 로그인 승인 대기 및 클릭
try:
    login_button = WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.CSS_SELECTOR, "button.btn_agree[name='user_oauth_approval'][value='true']")))
    login_button.click()
except Exception as e:
    print("로그인 버튼을 찾을 수 없거나 클릭할 수 없습니다:", e)

time.sleep(3)  # 로그인 이후 타임슬립 3초

data_list = []
save_count = 0

for index, row in df.iterrows():
    try:
        blog_url = row[0]
        blog_id = blog_url.split("/")[-1]  # URL에서 블로그 ID 추출
        url_2 = f"https://blogdex.space/blog-index/{blog_id}"
        
        driver.get(url_2)
        time.sleep(6)  # 아이디마다 크롤링할 때 타임슬립 4초

        # 필요한 항목들 스크래핑
        data = {
            "블로그명": driver.find_element(By.CSS_SELECTOR, "div.flex.space-x-1.pr-0.md\\:pr-24 p.text-sm.font-medium.leading-none").text,
            "블로그주소": f"https://blog.naver.com/{blog_id}",
            "블로그지수": driver.find_element(By.CSS_SELECTOR, "svg > text[font-family='Pretendard'][font-size='22px'][font-weight='700'][y='-60']").text,
            "블로그지수(확인용)": driver.find_element(By.CSS_SELECTOR, "div.flex.flex-1.justify-center.px-5 text:nth-child(2)").text,
            "블로그생성일": driver.find_element(By.CSS_SELECTOR, "div.w-full.space-x-1.pr-0.pt-4.md\\:pr-24 div.flex.items-center.justify-center.space-x-2.md\\:justify-start p.text-sm.font-medium.leading-none").text,
            "총방문자": driver.find_element(By.CSS_SELECTOR, "div.w-full.space-x-1.pb-8.pr-0.pt-4.md\\:pb-0.md\\:pr-24 div.flex.items-center.justify-center.space-x-2.md\\:justify-start p.text-sm.font-medium.leading-none").text,
            "총포스팅": driver.find_element(By.CSS_SELECTOR, "#__next > div.flex.min-h-screen.flex-col > main > div > div.flex.flex-col.gap-4 > div:nth-child(1) > div.p-6.pt-0 > div:nth-child(3) > div:nth-child(1) > div > div").text,
            "총구독자": driver.find_element(By.CSS_SELECTOR, "#__next > div.flex.min-h-screen.flex-col > main > div > div.flex.flex-col.gap-4 > div:nth-child(1) > div.p-6.pt-0 > div:nth-child(5) > div:nth-child(1) > div > div").text,
            "주제지수": driver.find_element(By.CSS_SELECTOR, "#__next > div.flex.min-h-screen.flex-col > main > div > div.flex.flex-col.gap-4 > div:nth-child(1) > div.p-6.pt-0 > div.flex.flex-col.justify-center.space-y-12.py-5.md\\:flex-row.md\\:justify-between.md\\:space-x-0.md\\:space-y-0.md\\:py-0 > div.divide.md\\:auto.flex.w-full.flex-1.flex-col.items-center.space-y-4.divide-y.px-5.text-center.md\\:items-end.md\\:text-right > div.pl-0.pt-8.md\\:pl-24.md\\:pt-0 > div > div > p").text,
            "종합지수": driver.find_element(By.CSS_SELECTOR, "#__next > div.flex.min-h-screen.flex-col > main > div > div.flex.flex-col.gap-4 > div:nth-child(1) > div.p-6.pt-0 > div.flex.flex-col.justify-center.space-y-12.py-5.md\\:flex-row.md\\:justify-between.md\\:space-x-0.md\\:space-y-0.md\\:py-0 > div.divide.md\\:auto.flex.w-full.flex-1.flex-col.items-center.space-y-4.divide-y.px-5.text-center.md\\:items-end.md\\:text-right > div:nth-child(2) > div > div > p").text,
            "최고지수": driver.find_element(By.CSS_SELECTOR, "#__next > div.flex.min-h-screen.flex-col > main > div > div.flex.flex-col.gap-4 > div:nth-child(1) > div.p-6.pt-0 > div.flex.flex-col.justify-center.space-y-12.py-5.md\\:flex-row.md\\:justify-between.md\\:space-x-0.md\\:space-y-0.md\\:py-0 > div.divide.md\\:auto.flex.w-full.flex-1.flex-col.items-center.space-y-4.divide-y.px-5.text-center.md\\:items-end.md\\:text-right > div:nth-child(3) > div > div > p").text,
            "블로그주제": driver.find_element(By.CSS_SELECTOR, "div.w-full.pt-4.md\\:w-auto.md\\:pt-0 div.flex.items-center.justify-center.space-x-2.md\\:justify-end p.text-sm.font-medium.leading-none").text,
            "블덱스전체랭킹": driver.find_element(By.CSS_SELECTOR, "#__next > div.flex.min-h-screen.flex-col > main > div > div.flex.flex-col.gap-4 > div:nth-child(1) > div.p-6.pt-0 > div:nth-child(5) > div:nth-child(3) > div > div > p").text,
            "블덱스주제랭킹": driver.find_element(By.CSS_SELECTOR, "div.ml-0.w-full.pt-4.md\\:ml-16.md\\:w-auto.md\\:pt-0 div.flex.items-center.space-x-2.justify-center.md\\:justify-center p.text-sm.font-medium.leading-none").text,
            "최적화수치": driver.find_element(By.CSS_SELECTOR, "div.relative.flex.rounded-md.w-9\\/10 div.bg-primary.h-6.rounded-l-md p.absolute.left-1\\/2.top-1\\/2").text,
            "메일주소": f"{blog_id}@naver.com"
        }
        data_list.append(data)
        save_count += 1

        # 실시간으로 출력
        print(f"아이디: {blog_id}")
        for key, value in data.items():
            print(f"{key}: {value}")
        print("\n")


        # 50개 단위로 중간 저장
        if save_count % 50 == 0:
            temp_df = pd.DataFrame(data_list)
            temp_save_path = f"{save_path.rsplit('.', 1)[0]}_temp_{save_count}.xlsx"
            temp_df.to_excel(temp_save_path, index=False)
            print(f"{save_count}개의 데이터가 {temp_save_path}에 중간 저장되었습니다.")

    except Exception as e:
        print(f"일부 요소를 찾을 수 없습니다: {e}")
        continue

# DataFrame으로 변환
result_df = pd.DataFrame(data_list)

# 최종 엑셀 파일로 저장
result_df.to_excel(save_path, index=False)

# 엑셀 파일 열기 및 서식 지정
wb = load_workbook(save_path)
ws = wb.active

# 열 너비 설정
column_widths = [18, 35, 12, 12, 14, 10, 9, 9, 9, 9, 9, 12, 20, 20, 12, 24]
for i, width in enumerate(column_widths, start=1):
    ws.column_dimensions[chr(64 + i)].width = width

# '최적' 포함된 셀 서식 변경
red_fill = PatternFill(start_color='FD1E19', end_color='FD1E19', fill_type='solid')
bold_font = Font(bold=True, color='FFFFFF')

for cell in ws['C']:
    if '최적' in str(cell.value):
        cell.font = bold_font
        cell.fill = red_fill

ws.auto_filter.ref = ws.dimensions

# 변경사항 저장
wb.save(save_path)

# 드라이버 종료
driver.quit()

print(f"모든 데이터가 {save_path}에 저장되었습니다.")


C:\Users\USER\AppData\Local\Temp\ipykernel_25296\1123688966.py:67: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  blog_url = row[0]


아이디: sooyeon1202
블로그명: 수여니(sooyeon1202)
블로그주소: https://blog.naver.com/sooyeon1202
블로그지수: 최적2+
블로그지수(확인용): 최적2+
블로그생성일: 2009-03-21
총방문자: 14,744,354
총포스팅: 1,992
총구독자: 6,515
주제지수: 최적1+
종합지수: 최적2+
최고지수: 최적2+
블로그주제: 패션·미용
블덱스전체랭킹: 46,587등(상위 5.0%)
블덱스주제랭킹: 1,881등(상위 5.9%)
최적화수치: 90.1%
메일주소: sooyeon1202@naver.com


아이디: myojinx
블로그명: 리뷰하는 묘대리(myojinx)
블로그주소: https://blog.naver.com/myojinx
블로그지수: 최적2+
블로그지수(확인용): 최적2+
블로그생성일: 2016-01-16
총방문자: 12,469,458
총포스팅: 1,853
총구독자: 13,892
주제지수: 최적2+
종합지수: 최적2+
최고지수: 최적3+
블로그주제: 패션·미용
블덱스전체랭킹: 12,102등(상위 1.3%)
블덱스주제랭킹: 452등(상위 1.4%)
최적화수치: 92.6%
메일주소: myojinx@naver.com


아이디: jihae2424
블로그명: 퓨어벨(jihae2424)
블로그주소: https://blog.naver.com/jihae2424
블로그지수: 최적2+
블로그지수(확인용): 최적2+
블로그생성일: 2009-03-13
총방문자: 2,261,655
총포스팅: 833
총구독자: 2,903
주제지수: 최적2+
종합지수: 최적2+
최고지수: 최적4+
블로그주제: 패션·미용
블덱스전체랭킹: 2,720등(상위 0.3%)
블덱스주제랭킹: 137등(상위 0.4%)
최적화수치: 92.9%
메일주소: jihae2424@naver.com


아이디: cubbobbo
블로그명: 쉬늉의 뷰티패션(cubbobbo)
블로그주소: https://blog.naver.com/cubbobbo
블로그지수: 최적2+
블로그